In [ ]:
!nvidia-smi

Tue Jun 15 05:30:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 4.6MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=7015baa6fbdf42ad605420384f34e70cbb4dfa258fd8d3a8807ec160df603bfa
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c deep-learning-hw2p2-part-2-classification

 96% 571M/594M [00:06<00:00, 107MB/s] 
100% 594M/594M [00:06<00:00, 93.3MB/s]


In [ ]:
!unzip \*.zip  && rm *.zip


Streaming output truncated to the last 5000 lines.
  inflating: val_data/val_data/2348/0001_01.jpg  
  inflating: val_data/val_data/2348/0046_01.jpg  
  inflating: val_data/val_data/2349/0082_01.jpg  
  inflating: val_data/val_data/2349/0312_02.jpg  
  inflating: val_data/val_data/235/0008_02.jpg  
  inflating: val_data/val_data/235/0140_03.jpg  
  inflating: val_data/val_data/2350/0106_01.jpg  
  inflating: val_data/val_data/2350/0339_01.jpg  
  inflating: val_data/val_data/2351/0006_01.jpg  
  inflating: val_data/val_data/2351/0150_01.jpg  
  inflating: val_data/val_data/2352/0193_01.jpg  
  inflating: val_data/val_data/2352/0311_01.jpg  
  inflating: val_data/val_data/2353/0229_01.jpg  
  inflating: val_data/val_data/2353/0297_02.jpg  
  inflating: val_data/val_data/2354/0140_01.jpg  
  inflating: val_data/val_data/2354/0444_01.jpg  
  inflating: val_data/val_data/2355/0112_02.jpg  
  inflating: val_data/val_data/2355/0385_01.jpg  
  inflating: val_data/val_data/2356/0097_01.jpg  
 

In [ ]:
#!unzip \*.zip  -d /content/drive/MyDrive/Homeworks/HW2P2

In [ ]:
import numpy as np
import torch
import PIL
import torchvision   

from torch.utils.data import Dataset, DataLoader

In [ ]:
import os
import numpy as np
from PIL import Image

import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [ ]:
import os
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def parse_data(datadir):
    img_list = []
    ID_list = []
    for root, directories, filenames in os.walk(datadir):  #root: median/1
        for filename in filenames:
            if filename.endswith('.jpg'):
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(root.split('/')[-1])
                
                # print(ID_list)

    # construct a dictionary, where key and value correspond to ID and target
    uniqueID_list = list(set(ID_list))
    class_n = len(uniqueID_list)
    target_dict = dict(zip(uniqueID_list, range(class_n)))
    target_num= dict(zip( range(class_n),uniqueID_list))
    print(target_num)
    print(target_dict)
    label_list = [target_dict[ID_key] for ID_key in ID_list]
    print(len(label_list))

    print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, label_list, class_n,target_num


In [ ]:
img_list, label_list, class_n,target_num = parse_data('/content/train_data/train_data')
print(len(img_list),len(label_list),class_n)

{0: '3500', 1: '1028', 2: '1438', 3: '329', 4: '1200', 5: '959', 6: '1878', 7: '3871', 8: '2158', 9: '3678', 10: '804', 11: '3623', 12: '433', 13: '930', 14: '68', 15: '2058', 16: '3991', 17: '2699', 18: '821', 19: '928', 20: '178', 21: '1977', 22: '3780', 23: '118', 24: '2535', 25: '3445', 26: '2283', 27: '1943', 28: '2894', 29: '269', 30: '3586', 31: '3454', 32: '3088', 33: '363', 34: '1114', 35: '3024', 36: '2818', 37: '2290', 38: '945', 39: '651', 40: '1881', 41: '13', 42: '1242', 43: '1510', 44: '578', 45: '547', 46: '2998', 47: '1795', 48: '1366', 49: '1263', 50: '2176', 51: '2020', 52: '1086', 53: '74', 54: '1029', 55: '2950', 56: '3704', 57: '2291', 58: '2728', 59: '1719', 60: '115', 61: '3182', 62: '1619', 63: '3969', 64: '3948', 65: '3988', 66: '3446', 67: '2483', 68: '318', 69: '2678', 70: '3651', 71: '1506', 72: '2495', 73: '1373', 74: '608', 75: '691', 76: '1756', 77: '2250', 78: '3892', 79: '3952', 80: '3311', 81: '1010', 82: '2466', 83: '1215', 84: '2434', 85: '3277', 86

In [ ]:
val_img_list, val_label_list, class_n,target_num = parse_data('/content/val_data/val_data')
# a_file = open("data.pkl", "wb")
# pickle.dump(target_num, a_file)
# a_file.close()
print(len(val_img_list),len(val_label_list),class_n)

{0: '3500', 1: '1028', 2: '1438', 3: '329', 4: '1200', 5: '959', 6: '1878', 7: '3871', 8: '2158', 9: '3678', 10: '804', 11: '3623', 12: '433', 13: '930', 14: '68', 15: '2058', 16: '3991', 17: '2699', 18: '821', 19: '928', 20: '178', 21: '1977', 22: '3780', 23: '118', 24: '2535', 25: '3445', 26: '2283', 27: '1943', 28: '2894', 29: '269', 30: '3586', 31: '3454', 32: '3088', 33: '363', 34: '1114', 35: '3024', 36: '2818', 37: '2290', 38: '945', 39: '651', 40: '1881', 41: '13', 42: '1242', 43: '1510', 44: '578', 45: '547', 46: '2998', 47: '1795', 48: '1366', 49: '1263', 50: '2176', 51: '2020', 52: '1086', 53: '74', 54: '1029', 55: '2950', 56: '3704', 57: '2291', 58: '2728', 59: '1719', 60: '115', 61: '3182', 62: '1619', 63: '3969', 64: '3948', 65: '3988', 66: '3446', 67: '2483', 68: '318', 69: '2678', 70: '3651', 71: '1506', 72: '2495', 73: '1373', 74: '608', 75: '691', 76: '1756', 77: '2250', 78: '3892', 79: '3952', 80: '3311', 81: '1010', 82: '2466', 83: '1215', 84: '2434', 85: '3277', 86

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, file_list, target_list):
        self.file_list = file_list
        self.target_list = target_list
        self.n_class = len(list(set(target_list)))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        label = self.target_list[index]
        return img, label

In [ ]:
trainset = ImageDataset(img_list, label_list)

In [ ]:
valset= ImageDataset(val_img_list,val_label_list)

In [ ]:
dataloader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=1, drop_last=False)

In [ ]:
dev_dataloader= DataLoader(valset, batch_size=256, shuffle=False, num_workers=1)

In [ ]:
train_data_item, train_data_label = trainset.__getitem__(23)
print(train_data_label,train_data_item)

2536 tensor([[[0.2392, 0.2235, 0.2039,  ..., 0.1059, 0.0980, 0.1490],
         [0.2353, 0.2235, 0.2039,  ..., 0.1098, 0.0941, 0.1176],
         [0.2353, 0.2275, 0.2118,  ..., 0.1216, 0.1020, 0.0902],
         ...,
         [0.0627, 0.0510, 0.0745,  ..., 0.5804, 0.5765, 0.5569],
         [0.0745, 0.0627, 0.0745,  ..., 0.5373, 0.5647, 0.6118],
         [0.0784, 0.0745, 0.0745,  ..., 0.6196, 0.6078, 0.5765]],

        [[0.1059, 0.0941, 0.0824,  ..., 0.0510, 0.0431, 0.0941],
         [0.1059, 0.0941, 0.0902,  ..., 0.0549, 0.0392, 0.0627],
         [0.1137, 0.1059, 0.1059,  ..., 0.0667, 0.0471, 0.0353],
         ...,
         [0.0627, 0.0510, 0.0745,  ..., 0.5569, 0.5529, 0.5333],
         [0.0745, 0.0627, 0.0745,  ..., 0.5176, 0.5451, 0.5922],
         [0.0784, 0.0745, 0.0745,  ..., 0.5882, 0.5765, 0.5451]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0078, 0.0078, 0.0588],
         [0.0000, 0.0000, 0.0000,  ..., 0.0118, 0.0039, 0.0275],
         [0.0000, 0.0000, 0.0000,  ..., 0.0235, 0.011

In [ ]:
print('data item shape: {}\t data item label: {}'.format(train_data_item.shape, train_data_label))

data item shape: torch.Size([3, 64, 64])	 data item label: 2536


In [ ]:
class SimpleResidualBlock(nn.Module):
    def __init__(self, channel_size,channel_next, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(channel_size, channel_next, kernel_size=3,padding=1, stride=stride, bias=False)      
        self.bn1 = nn.BatchNorm2d(channel_next)
        if stride == 1:
            self.shortcut = nn.Identity()
        else:
            self.shortcut = nn.Conv2d(channel_size, channel_next, kernel_size=1, stride=stride)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        
        shortcut = self.shortcut(x)
        
        out = self.relu(out + shortcut)
        
        return out


In [ ]:
class ClassificationNetwork(nn.Module):
    def __init__(self, in_features, num_classes):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Conv2d(in_features, 64, kernel_size=3, stride=1,padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            SimpleResidualBlock(64,64,1),
            SimpleResidualBlock(64,128,2),
            SimpleResidualBlock(128,128,1),
            SimpleResidualBlock(128,256,2),
            SimpleResidualBlock(256,256,1),
            SimpleResidualBlock(256,512,2),
            SimpleResidualBlock(512,512,1),
            nn.AdaptiveAvgPool2d((1, 1)), # For each channel, collapses (averages) the entire feature map (height & width) to 1x1
            nn.Flatten(), # the above ends up with batch_size x 64 x 1 x 1, flatten to batch_size x 64
        )
        self.linear = nn.Linear(512, num_classes)
        
    def forward(self, x, return_embedding=False):
        embedding = self.layers(x) 
        
        if return_embedding:
            return embedding
        else:
            return self.linear(embedding)


In [ ]:
numEpochs = 10
in_features = 3 # RGB channels

learningRate = 5e-2
weightDecay = 5e-5

num_classes = 4000

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network = ClassificationNetwork(in_features, num_classes)
network = network.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
print(network)

ClassificationNetwork(
  (layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): SimpleResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Identity()
      (relu): ReLU()
    )
    (4): SimpleResidualBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2))
      (relu): ReLU()
    )
    (5): SimpleResidualBlock(
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0

In [ ]:
# Train!

def train_epoch(network,dataloader,criterion,optimizer,epoch):
    # Train
    network.train()
    avg_loss = 0.0
    num_correct=0
    total_predictions=0
    accuracy=0
    num_total=0
    running_loss=0
    accur=[]

    for batch_num, (x, y) in enumerate(dataloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = network(x)

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()
        value,index=torch.max(outputs,1)
        num_correct=torch.sum(index==y)
        total_predictions=outputs.shape[0]
        # print("Number of correct predictions",num_correct)
        num_total=num_total+num_correct
        batch_accuracy=num_correct/256
        # print(batch_num,"bactch accuracy is:",batch_accuracy)
        # running_loss/=len(dataloader)
        # print("Training Loss:", running_loss)
    


        # if batch_num % 10 == 9:
        #     print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, avg_loss/))
        #     avg_loss = 0.0
    
    # # Validate
    # network.eval()
    # num_correct = 0
    # for batch_num, (x, y) in enumerate(dev_dataloader):
    #     x, y = x.to(device), y.to(device)
    #     outputs = network(x)
    #     num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
        
    # print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch, num_correct / len(dev_dataset)))
    print("*****************************************========================================************++++++++++++++")
    accuracy=num_total/380638
    accur.append(accuracy)
    print(epoch,"epoch:",accuracy)
    path = '/content/drive/MyDrive/Image_classification/Sunday_13-06-2021_CNN_model'+str(epoch)
    torch.save(network.state_dict(), path)
    # Validate
    



In [ ]:


def val_epoch(network,dev_dataloader):
  network.eval()
  avg_loss = 0.0
  num_correct=0
  total_predictions=0
  accuracy=0
  num_total=0
  running_loss=0
  accur=[]

  for batch_num, (x, y) in enumerate(dev_dataloader):
      
      x, y = x.to(device), y.to(device)

      outputs = network(x)

    
      # print(y)

      
      value,index=torch.max(outputs,1)
      num_correct=torch.sum(index==y)
      # print(y)
      # print(index)
      total_predictions=outputs.shape[0]
      print("Number of correct predictions",num_correct)
      num_total=num_total+num_correct
      batch_accuracy=num_correct/256
      # print(batch_num,"bactch accuracy is:",batch_accuracy)
      # running_loss/=len(dataloader)
      # print("Training Loss:", running_loss)



      # if batch_num % 10 == 9:
      #     print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, avg_loss/))
      #     avg_loss = 0.0

  # # Validate
  # network.eval()
  # num_correct = 0
  # for batch_num, (x, y) in enumerate(dev_dataloader):
  #     x, y = x.to(device), y.to(device)
  #     outputs = network(x)
  #     num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
      
  # print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch, num_correct / len(dev_dataset)))
  print("*****************************************========================================************++++++++++++++")
  accuracy=num_total/8000
  print("val accuracy is:",accuracy)
  # accur.append(accuracy)
  # print(epoch,"epoch:",accuracy)
  # torch.save(network.state_dict(), path)
  # # Validate




In [ ]:

for epoch in range(22,25):
  train_epoch(network,dataloader,criterion,optimizer,epoch)
  val_epoch(network,dev_dataloader)

*****************************************========================================************++++++++++++++
22 epoch: tensor(0.9310, device='cuda:0')
Number of correct predictions tensor(154, device='cuda:0')
Number of correct predictions tensor(145, device='cuda:0')
Number of correct predictions tensor(149, device='cuda:0')
Number of correct predictions tensor(149, device='cuda:0')
Number of correct predictions tensor(161, device='cuda:0')
Number of correct predictions tensor(162, device='cuda:0')
Number of correct predictions tensor(141, device='cuda:0')
Number of correct predictions tensor(146, device='cuda:0')
Number of correct predictions tensor(156, device='cuda:0')
Number of correct predictions tensor(161, device='cuda:0')
Number of correct predictions tensor(147, device='cuda:0')
Number of correct predictions tensor(157, device='cuda:0')
Number of correct predictions tensor(148, device='cuda:0')
Number of correct predictions tensor(150, device='cuda:0')
Number of correct predic

In [ ]:
#  loading the model.

path='/content/drive/MyDrive/Image_classification/Sunday_13-06-2021_CNN_model24'
state_dict = torch.load(path)
#print(len(state_dict["net.14.weight"]))
network.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
# val_epoch(network,dev_dataloader)

In [ ]:
# state_dict = torch.load(path,map_location=torch.device('cpu'))
# #print(len(state_dict["net.14.weight"]))
# network.load_state_dict(state_dict)


In [ ]:

def parse_data(datadir):
    img_list = []
    ID_list = []
    for root, directories, filenames in os.walk(datadir):  #root: median/1
        for filename in filenames:
            if filename.endswith('.jpg'):
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(root.split('/')[-1])
                # print(ID_list)

    # construct a dictionary, where key and value correspond to ID and target
    # uniqueID_list = list(set(ID_list))
    # class_n = len(uniqueID_list)
    # target_dict = dict(zip(uniqueID_list, range(class_n)))
    # print(target_dict)
    # label_list = [target_dict[ID_key] for ID_key in ID_list]
    # print(len(label_list))
    class_n=4000

    # print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, class_n


In [ ]:
# from PIL import Image


# img=Image.open('/content/val_data/val_data/3120/0001_01.jpg')
# plt.imshow(img)


class ImageDatatest(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list
        # self.target_list = target_list
        # self.n_class = len(list(set(target_list)))
        self.n_class=4000

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        # label = self.target_list[index]
        return img



In [ ]:
test_img_list, class_n = parse_data('/content/test_data/test_data')
testset = ImageDatatest(test_img_list)
test_dataloader= DataLoader(testset, batch_size=256, shuffle=False, num_workers=1)
test_data_item= testset.__getitem__(23)
print(target_num)

{0: '3500', 1: '1028', 2: '1438', 3: '329', 4: '1200', 5: '959', 6: '1878', 7: '3871', 8: '2158', 9: '3678', 10: '804', 11: '3623', 12: '433', 13: '930', 14: '68', 15: '2058', 16: '3991', 17: '2699', 18: '821', 19: '928', 20: '178', 21: '1977', 22: '3780', 23: '118', 24: '2535', 25: '3445', 26: '2283', 27: '1943', 28: '2894', 29: '269', 30: '3586', 31: '3454', 32: '3088', 33: '363', 34: '1114', 35: '3024', 36: '2818', 37: '2290', 38: '945', 39: '651', 40: '1881', 41: '13', 42: '1242', 43: '1510', 44: '578', 45: '547', 46: '2998', 47: '1795', 48: '1366', 49: '1263', 50: '2176', 51: '2020', 52: '1086', 53: '74', 54: '1029', 55: '2950', 56: '3704', 57: '2291', 58: '2728', 59: '1719', 60: '115', 61: '3182', 62: '1619', 63: '3969', 64: '3948', 65: '3988', 66: '3446', 67: '2483', 68: '318', 69: '2678', 70: '3651', 71: '1506', 72: '2495', 73: '1373', 74: '608', 75: '691', 76: '1756', 77: '2250', 78: '3892', 79: '3952', 80: '3311', 81: '1010', 82: '2466', 83: '1215', 84: '2434', 85: '3277', 86

In [ ]:
def test_epoch(network,test_dataloder):
  
  network.eval()
  avg_loss = 0.0
  num_correct=0
  total_predictions=0
  accuracy=0
  num_total=0
  running_loss=0
  accur=[]
  result=[]


  for batch_idx, data in enumerate(test_dataloader):

    data=data.to(device)
    outputs=network(data) 
    # print(outputs.shape)
    value,index=torch.max(outputs,1)
    result.append(index.tolist())
  return result 
      

      
      





In [ ]:
result=test_epoch(network,test_dataloader)

In [ ]:
images=[]
for i in range(len(test_img_list)):
  images.append(test_img_list[i].split("/")[-1])
print(images)

['5484.jpg', '1743.jpg', '3690.jpg', '6091.jpg', '1426.jpg', '7276.jpg', '4988.jpg', '7261.jpg', '2068.jpg', '7947.jpg', '5936.jpg', '2510.jpg', '2983.jpg', '3725.jpg', '2882.jpg', '5589.jpg', '840.jpg', '1644.jpg', '7801.jpg', '4762.jpg', '5879.jpg', '1803.jpg', '2889.jpg', '5115.jpg', '19.jpg', '7912.jpg', '1050.jpg', '6608.jpg', '1389.jpg', '3373.jpg', '2439.jpg', '7229.jpg', '6466.jpg', '7579.jpg', '3169.jpg', '1889.jpg', '2170.jpg', '3662.jpg', '2967.jpg', '2368.jpg', '1368.jpg', '2656.jpg', '1691.jpg', '3576.jpg', '6233.jpg', '5548.jpg', '7622.jpg', '5060.jpg', '6989.jpg', '3922.jpg', '4160.jpg', '1160.jpg', '6048.jpg', '1606.jpg', '6674.jpg', '5962.jpg', '5408.jpg', '4180.jpg', '1061.jpg', '1208.jpg', '7954.jpg', '6144.jpg', '667.jpg', '2492.jpg', '70.jpg', '7994.jpg', '1942.jpg', '1069.jpg', '4986.jpg', '7566.jpg', '4233.jpg', '7578.jpg', '971.jpg', '3522.jpg', '1412.jpg', '7153.jpg', '651.jpg', '898.jpg', '4140.jpg', '211.jpg', '6163.jpg', '4367.jpg', '6882.jpg', '6867.jpg', '

In [ ]:
tensor_result=torch.as_tensor(result[0:-1]).flatten()
tensor_result.shape
last=torch.as_tensor(result[-1])
final_tensor=torch.hstack((tensor_result,last))
final=final_tensor.tolist()
from pandas import DataFrame
# your_list = ['item1', 'item2', 'item3',...]
# df = DataFrame (images,final,columns=['id','labels'])
df = DataFrame(final, index =images,columns =['label'])
df.to_csv("result12345.csv",index_label='id')
df.head


<bound method NDFrame.head of           label
5484.jpg    408
1743.jpg   2445
3690.jpg    508
6091.jpg    525
1426.jpg   3502
...         ...
7028.jpg   2920
474.jpg     604
777.jpg    1045
925.jpg    1534
145.jpg    1983

[8000 rows x 1 columns]>

In [ ]:
# import the csv file
import pandas as pd
file=pd.read_csv('/content/result123456.csv')
# print(file.head)
li=[]
news=[]
# print("The images are",file['id'])
li=file['label']
# print(target_num)
for i in li:
  news.append((target_num[i]))
# print(news)
file['label']=news
file.to_csv('h123456.csv')
file.head(5)



,id,label
0,0.jpg,3771
1,1.jpg,2433
2,10.jpg,785
3,100.jpg,1680
4,1000.jpg,627


In [ ]:
!kaggle competitions submit -c deep-learning-hw2p2-part-2-classification -f h123456.csv -m "Third"

100% 114k/114k [00:04<00:00, 26.5kB/s]
Successfully submitted to Deep Learning HW2P2 Part 2 (Classification))